- dashboard
- dash gallery app: F1 analytics, soccer data analysis (rider stats, strenghts, etc.), basketball shot explorer, course attack prediction based on wind/gradient/speciality

In [1]:
import dash
from dash import html
from dash import dcc
import plotly.graph_objects as go
import plotly.express as px
from dash.dependencies import Input, Output

import pandas as pd
import glob
import re
import collections

# Load Data

In [2]:
rider_names = list(pd.read_csv('../data/pcs-scraping/rider_names.csv'))

In [3]:
#rider_name = rider_names[0]
rider_name = 'tadej-pogačar'

path = f'../data/pcs-scraping/results/rider/{rider_name}'

In [4]:
rider_names_label = [" ".join(x.split("-")).title() for x in rider_names]
year_label = sorted([re.search('\d{4}', file).group(0) for file in glob.glob(f'{path}/*_clean.csv')], reverse=True)

In [5]:
df_rider_clean = {rider_name: {}}

for file in glob.glob(f'{path}/*_clean.csv'):

    year = re.search('\d{4}', file).group(0)
    df_rider_clean[rider_name][year] = pd.read_csv(file)

# Dashboard App with Tabs

In [ ]:
app.layout = html.Div([
    html.H1('Dash Tabs component demo'),
    dcc.Tabs(id="tabs-example-graph", value='tab-1-example-graph', children=[
        dcc.Tab(label='Tab One', value='tab-1-example-graph'),
        dcc.Tab(label='Tab Two', value='tab-2-example-graph'),
    ]),
    html.Div(id='tabs-content-example-graph')
])

In [ ]:
@app.callback(Output('tabs-content-example-graph', 'children'),
              Input('tabs-example-graph', 'value'))
def render_content(tab):
    if tab == 'tab-1-example-graph':
        
        return html.Div([
            html.H3('Tab content 1'),
            dcc.Graph(
                id='graph-1-tabs',
                figure={
                    'data': [{
                        'x': [1, 2, 3],
                        'y': [3, 1, 2],
                        'type': 'bar'
                    }]
                }
            )
        ])
    elif tab == 'tab-2-example-graph':
        
        return html.Div([
            html.H3('Tab content 2'),
            dcc.Graph(
                id='graph-2-tabs',
                figure={
                    'data': [{
                        'x': [1, 2, 3],
                        'y': [5, 10, 6],
                        'type': 'bar'
                    }]
                }
            )
        ])

In [ ]:
def graph_tab1():
    
    pass


def graph_tab2():
    
    pass

In [ ]:
@app.callback(Output('tabs-content-example-graph', 'children'),
              Input('tabs', 'value'))
def render_content(tab):
    
    if tab == 'season_results':
        
        return graph_tab1()
    
    elif tab == 'career_results':
        
        return graph_tab2()

In [ ]:
app = dash.Dash()

app.layout = html.Div([html.Div(id = 'parent',
                      children = [html.H1(id = 'H1',
                                          children = 'Rider Dashboard',
                                          style = {'textAlign':'center',
                                                   'marginTop':40,'marginBottom':40}),

                                  dcc.Dropdown(id = 'rider_name',
                                               options = [{'label': rider_name_label, 'value': rider_name} for rider_name, rider_name_label in zip(rider_names, rider_names_label)],
                                               value = rider_names[0]),
                                  dcc.Dropdown(id = 'year',
                                               options = [{'label': year, 'value': year} for year in year_label],
                                               value = year_label[0]),
                                  dcc.Tabs(id='tabs', value='season_results', children=[
                                      dcc.Tab(label='Season Results', value='season_results'),
                                      dcc.Tab(label='Career Results', value='career_results')
                                  ])
                                 ])
                      ])
    
    
################## Utility Functions #####################



###########################################################
        

################## App Functions ###########################


In [ ]:
if __name__ == '__main__': 
    app.run_server()

# Dashboard App without Tabs

In [12]:
app = dash.Dash()

app.layout = html.Div([html.Div(id = 'parent',
                      children = [html.H1(id = 'H1',
                                          children = 'Rider Dashboard',
                                          style = {'textAlign':'center',
                                                   'marginTop':40,'marginBottom':40}),

                                  dcc.Dropdown(id = 'rider_name',
                                               options = [{'label': rider_name_label, 'value': rider_name} for rider_name, rider_name_label in zip(rider_names, rider_names_label)],
                                               value = rider_names[0]),
                                  dcc.Dropdown(id = 'year',
                                               options = [{'label': year, 'value': year} for year in year_label],
                                               value = year_label[0]),
                                  html.Div(id='tabs_content'),
                                  dcc.Graph(id = 'results_plot_date'),
                                  dcc.Graph(id = 'results_plot_race'),
                                  dcc.Graph(id = 'results_plot_stats'), 
                                  html.Div(children=[
                                      dcc.Graph(id="results_plot_racetype", style={'display': 'inline-block'}),
                                      dcc.Graph(id="results_plot_racetype_mean", style={'display': 'inline-block'})]),
                                  html.Div(children=[
                                      dcc.Graph(id="results_plot_stats_wins", style={'display': 'inline-block'}),
                                      dcc.Graph(id="results_plot_stats_five", style={'display': 'inline-block'}), 
                                      dcc.Graph(id="results_plot_stats_ten", style={'display': 'inline-block'}), 
                                      dcc.Graph(id="results_plot_stats_twenty", style={'display': 'inline-block'})]), 
                                  html.Div(children=[
                                      dcc.Graph(id="results_plot_racedays", style={'display': 'inline-block'}),
                                      dcc.Graph(id="results_plot_racedays_by_racetype", style={'display': 'inline-block'})])
                                 ])
                      ])
    
    
################## Utility Functions #####################
def getRiderData(rider_name, year):
    
    # get rider cleaned data
    path = f'../data/pcs-scraping/results/rider/{rider_name}'
    df_rider_clean = {rider_name: {}}

    for file in glob.glob(f'{path}/*_clean.csv'):

        year_ = re.search('\d{4}', file).group(0)
        df_rider_clean[rider_name][year_] = pd.read_csv(file)
        
    return df_rider_clean

def getRiderStats(df_rider_clean, rider_name, year):
    
    rider_top_stats = collections.defaultdict(dict)

    for year, df in df_rider_clean[rider_name].items():

        rider_top_stats[year]['wins'] = (df['Result'] == 1).sum()
        rider_top_stats[year]['top_five'] = (df['Result'] <= 5).sum()
        rider_top_stats[year]['top_ten'] = (df['Result'] <= 10).sum()
        
    return rider_top_stats

def getResultsByRaceType(df_rider_clean, rider_name, year):
    
    results_by_racetype = pd.DataFrame(df_rider_clean[rider_name][year][['Result', 'SubRaceType']].groupby('SubRaceType')
             , columns=['SubRaceType', 'Result']).set_index('SubRaceType')

    results_by_racetype = results_by_racetype['Result'].apply(lambda x: x['Result'])
    
    return results_by_racetype

def getRiderStatsByRaceType(df_rider_clean, rider_name, year):
    
    wins = df_rider_clean[rider_name][year][df_rider_clean[rider_name][year]['Result']==1]
    top_five = df_rider_clean[rider_name][year][df_rider_clean[rider_name][year]['Result']<=5]
    top_ten = df_rider_clean[rider_name][year][df_rider_clean[rider_name][year]['Result']<=10]
    top_twenty = df_rider_clean[rider_name][year][df_rider_clean[rider_name][year]['Result']<=20]

    return wins, top_five, top_ten, top_twenty

def getRiderRacedays(df_rider_clean, rider_name, year):
    
    racedays = {}

    for year, df in df_rider_clean[rider_name].items():

        racedays[year] = len(df)
        
    return racedays

def getRiderRacedaysByRacetype(df_rider_clean, rider_name, year):
    
    racedays_subracetype = {}

    for year, df in df_rider_clean[rider_name].items():

        racedays_subracetype[year] = df_rider_clean[rider_name][year].groupby('SubRaceType')['Result'].size()
        
    racedays_subracetype = pd.DataFrame(racedays_subracetype).T.sort_index()
    
    return racedays_subracetype
    

###########################################################
        

################## App Functions ###########################
@app.callback(Output(component_id='results_plot_date', component_property= 'figure'),
              [Input(component_id='rider_name', component_property= 'value'),
               Input(component_id='year', component_property= 'value')])
def graphRiderResultsDate(rider_name, year):    
    
    df_rider_clean = getRiderData(rider_name, year)
    
    # plot results
    if year in df_rider_clean[rider_name].keys():
    
        fig = go.Figure([go.Scatter(x = df_rider_clean[rider_name][year]['Date'],
                                    y = df_rider_clean[rider_name][year]['Result'],
                                    line = dict(color = 'firebrick', width = 4))
                        ])

        fig.update_layout(title = f'{rider_name} - {year}',
                          xaxis_title = 'Date',
                          yaxis_title = 'Result'
                          )
    # if no results for selected year, plot no data figure
    else:
        
        fig = go.Figure()
        fig.update_layout(
            xaxis =  { "visible": False },
            yaxis = { "visible": False },
            annotations = [
                {   
                    "text": "No Matching Data Found",
                    "xref": "paper",
                    "yref": "paper",
                    "showarrow": False,
                    "font": {
                        "size": 28
                    }
                }
            ]
        )
        
    return fig

@app.callback(Output(component_id='results_plot_race', component_property= 'figure'),
              [Input(component_id='rider_name', component_property= 'value'),
               Input(component_id='year', component_property= 'value')])
def graphRiderResultsRace(rider_name, year):    
    
    df_rider_clean = getRiderData(rider_name, year)
    
    # plot results
    if year in df_rider_clean[rider_name].keys():
    
        fig = go.Figure([go.Scatter(x = df_rider_clean[rider_name][year]['Race'],
                                    y = df_rider_clean[rider_name][year]['Result'],
                                    line = dict(color = 'firebrick', width = 4))
                        ])

        fig.update_layout(title = f'{rider_name} - {year}',
                          xaxis_title = 'Race',
                          yaxis_title = 'Result'
                          )
    # if no results for selected year, plot no data figure
    else:
        
        fig = go.Figure()
        fig.update_layout(
            xaxis =  { "visible": False },
            yaxis = { "visible": False },
            annotations = [
                {   
                    "text": "No Matching Data Found",
                    "xref": "paper",
                    "yref": "paper",
                    "showarrow": False,
                    "font": {
                        "size": 28
                    }
                }
            ]
        )
        
    return fig

@app.callback(Output(component_id='results_plot_stats', component_property= 'figure'),
              [Input(component_id='rider_name', component_property= 'value'),
               Input(component_id='year', component_property= 'value')])
def graphRiderStats(rider_name, year):    
    
    df_rider_clean = getRiderData(rider_name, year)
    rider_top_stats = getRiderStats(df_rider_clean, rider_name, year)
    
    # plot results
    if year in df_rider_clean[rider_name].keys():
    
        df_plot = pd.melt(pd.DataFrame(rider_top_stats).T.reset_index(), id_vars=['index'], value_vars=['wins', 'top_five', 'top_ten'])
        fig = px.bar(df_plot.sort_values(by=['index', 'variable']), x='index', y='value', color='variable', barmode='group')

        fig.update_layout(title = f'{rider_name} - Top Stats',
                          xaxis_title = 'Year',
                          yaxis_title = 'Count'
                          )
    # if no results for selected year, plot no data figure
    else:
        
        fig = go.Figure()
        fig.update_layout(
            xaxis =  { "visible": False },
            yaxis = { "visible": False },
            annotations = [
                {   
                    "text": "No Matching Data Found",
                    "xref": "paper",
                    "yref": "paper",
                    "showarrow": False,
                    "font": {
                        "size": 28
                    }
                }
            ]
        )
        
    return fig

@app.callback(Output(component_id='results_plot_racetype', component_property= 'figure'),
              [Input(component_id='rider_name', component_property= 'value'),
               Input(component_id='year', component_property= 'value')])
def graphResultsByRacetype(rider_name, year):    
    
    df_rider_clean = getRiderData(rider_name, year)
    results_by_racetype = getResultsByRaceType(df_rider_clean, rider_name, year)
    
    # plot results
    if year in df_rider_clean[rider_name].keys():
    
        fig = px.bar(results_by_racetype, barmode='group')
        
        fig.update_layout(title = f'{rider_name} - Results for {year}',
                          xaxis_title = 'RaceType',
                          yaxis_title = 'Result',
                          showlegend=False
                          )
        

    # if no results for selected year, plot no data figure
    else:
        
        fig = go.Figure()
        fig.update_layout(
            xaxis =  { "visible": False },
            yaxis = { "visible": False },
            annotations = [
                {   
                    "text": "No Matching Data Found",
                    "xref": "paper",
                    "yref": "paper",
                    "showarrow": False,
                    "font": {
                        "size": 28
                    }
                }
            ]
        )
        
    return fig

@app.callback(Output(component_id='results_plot_racetype_mean', component_property= 'figure'),
              [Input(component_id='rider_name', component_property= 'value'),
               Input(component_id='year', component_property= 'value')])
def graphResultsByRacetypeMean(rider_name, year):    
    
    df_rider_clean = getRiderData(rider_name, year)
    results_by_racetype = getResultsByRaceType(df_rider_clean, rider_name, year)
    
    # plot results
    if year in df_rider_clean[rider_name].keys():
    
        fig = px.bar(results_by_racetype.mean(axis=1))
        
        fig.update_layout(title = f'{rider_name} - Mean Results for {year}',
                          xaxis_title = 'RaceType',
                          yaxis_title = 'Mean Result',
                          showlegend=False
                          )
        

    # if no results for selected year, plot no data figure
    else:
        
        fig = go.Figure()
        fig.update_layout(
            xaxis =  { "visible": False },
            yaxis = { "visible": False },
            annotations = [
                {   
                    "text": "No Matching Data Found",
                    "xref": "paper",
                    "yref": "paper",
                    "showarrow": False,
                    "font": {
                        "size": 28
                    }
                }
            ]
        )
        
    return fig

@app.callback([Output(component_id='results_plot_stats_wins', component_property= 'figure'),
               Output(component_id='results_plot_stats_five', component_property= 'figure'), 
               Output(component_id='results_plot_stats_ten', component_property= 'figure'),
               Output(component_id='results_plot_stats_twenty', component_property= 'figure')],
              [Input(component_id='rider_name', component_property= 'value'),
               Input(component_id='year', component_property= 'value')])
def graphRiderStatsByRacetype(rider_name, year):    
    
    df_rider_clean = getRiderData(rider_name, year)
    wins, top_five, top_ten, top_twenty = getRiderStatsByRaceType(df_rider_clean, rider_name, year)
    
    if year in df_rider_clean[rider_name].keys():
        
        # plot results
        fig1 = px.bar(wins.groupby('SubRaceType')['Result'].count())

        fig1.update_layout(title = f'{rider_name} - Wins for {year}',
                              xaxis_title = 'RaceType',
                              yaxis_title = '# Wins',
                              showlegend=False
                              )

        fig2 = px.bar(top_five.groupby('SubRaceType')['Result'].count())

        fig2.update_layout(title = f'{rider_name} - Top 5 for {year}',
                              xaxis_title = 'RaceType',
                              yaxis_title = '# Top 5',
                              showlegend=False
                              )

        fig3 = px.bar(top_ten.groupby('SubRaceType')['Result'].count())

        fig3.update_layout(title = f'{rider_name} - Top 10 for {year}',
                              xaxis_title = 'RaceType',
                              yaxis_title = '# Top 10',
                              showlegend=False
                              )

        fig4 = px.bar(top_twenty.groupby('SubRaceType')['Result'].count())

        fig4.update_layout(title = f'{rider_name} - Top 20 for {year}',
                              xaxis_title = 'RaceType',
                              yaxis_title = '# Top 20',
                              showlegend=False
                              )
    
    # if no results for selected year, plot no data figure
    else:
        
        fig = go.Figure()
        fig.update_layout(
            xaxis =  { "visible": False },
            yaxis = { "visible": False },
            annotations = [
                {   
                    "text": "No Matching Data Found",
                    "xref": "paper",
                    "yref": "paper",
                    "showarrow": False,
                    "font": {
                        "size": 28
                    }
                }
            ]
        )
        
    return fig1, fig2, fig3, fig4

@app.callback([Output(component_id='results_plot_racedays', component_property= 'figure'),
               Output(component_id='results_plot_racedays_by_racetype', component_property= 'figure')],
              [Input(component_id='rider_name', component_property= 'value'),
               Input(component_id='year', component_property= 'value')])
def graphRacedays(rider_name, year):    
    
    df_rider_clean = getRiderData(rider_name, year)
    racedays = getRiderRacedays(df_rider_clean, rider_name, year)
    racedays_by_racetype = getRiderRacedaysByRacetype(df_rider_clean, rider_name, year)
    
    if year in df_rider_clean[rider_name].keys():
        
        # plot results
        fig1 = px.bar(pd.DataFrame.from_dict(racedays, columns=['racedays'], orient='index').sort_index())

        fig1.update_layout(title = f'{rider_name} - Racedays per Season',
                              xaxis_title = 'Season',
                              yaxis_title = '# Racedays',
                              showlegend=False
                              )

        fig2 = px.bar(racedays_by_racetype, barmode='stack')
        
        fig2.add_trace(go.Scatter(x=racedays_by_racetype.apply(lambda x: x/2).sum(axis=1).index,
                         y=racedays_by_racetype.apply(lambda x: x/2).sum(axis=1).values,
                         line=go.scatter.Line(color="purple"),
                         name='trend')
              )
        
        fig2.update_layout(title = f'{rider_name} - Racedays by Racetype per Season',
                              xaxis_title = 'Season',
                              yaxis_title = '# Racedays',
                              showlegend=True
                              )
    
    # if no results for selected year, plot no data figure
    else:
        
        fig = go.Figure()
        fig.update_layout(
            xaxis =  { "visible": False },
            yaxis = { "visible": False },
            annotations = [
                {   
                    "text": "No Matching Data Found",
                    "xref": "paper",
                    "yref": "paper",
                    "showarrow": False,
                    "font": {
                        "size": 28
                    }
                }
            ]
        )
        
    return fig1, fig2


In [13]:
if __name__ == '__main__': 
    app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)


- create tabs for:
    - current season results
    - career results
    - one day race results
    - stage race results